<a href="https://colab.research.google.com/github/ljkrajewski/jupyter_notebooks/blob/main/tic_tac_toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import pickle
import time
from os.path import exists

tablePath = '/content/3t_table_2.pkl'

In [ ]:
class tictactoeBoard:
    """
    The board structure:
        1  2  3   |   (x for 2^x [player locations])
     A  .  .  .   |   0  1  2
     B  .  .  .   |   3  4  5
     C  .  .  .   |   6  7  8

    square: 2 char string w/ board coords (eg, 'B3')
    board: 2 integer-element list where each element is the sum of player placements ([X,O])

    """

    def __init__(self):
        self.board = [0,0]  # [Xs occupied squares, Os occupied squares]

    def squareToInt(self,square):
        x = ord(square[1]) - 49 #1
        y = ord(square[0].upper()) - 65 #A
        ans = 2 ** (y*3 + x)
        return ans

    def isOccupied(self,squareInt):
        #squareInt = self.squareToInt(square)
        wholeBoard = self.board[0] + self.board[1]
        return (wholeBoard & squareInt) > 0

    def makeMoveHuman(self,player,square):  #used by human (square defined by letter/number coordinates)
        squareInt = self.squareToInt(square)
        self.makeMoveComputer(player,squareInt)

    def makeMoveComputer(self,player,squareInt):  #used by computer (square defined by powers of 2)
        playerX = self.board[0]
        playerO = self.board[1]
        #squareInt = self.squareToInt(square)
        if not self.isOccupied(squareInt):
            if (player.upper() == 'X'):
                playerX = playerX + squareInt
            else:
                playerO = playerO + squareInt
        else:
            print(square.upper() + " is occupied.")
        self.board = [playerX, playerO]

    def isWin(self,player):
        if player.upper() == "X":
            testPlayer = self.board[0]
        else:
            testPlayer = self.board[1]
        winningNums = [7,56,448,73,146,292,273,84]
        isWinner = False
        for x in winningNums:
            isWinner = isWinner or ((testPlayer & x) == x)
        return isWinner

    def isDraw(self):
        return (self.board[0] + self.board[1]) == 511

    def availableSquares(self):
        answer = []
        wholeBoard = self.board[0] + self.board[1]
        for x in range(0,9):
            squareInt = 2**x
            if (wholeBoard & squareInt) == 0:
                answer += [squareInt]
        return answer

    def printBoard(self):
        print("   1  2  3")
        for y in (0,1,2):
            print(chr(y+65) + "- ", end="")
            for x in (0,1,2):
                squareInt = 2 ** (y*3 + x)
                if (self.board[0] & squareInt) > 0:
                    print("X  ", end="")
                elif (self.board[1] & squareInt) > 0:
                    print("O  ", end="")
                else:
                    print(".  ", end="")
            print("")
        print("")

In [ ]:
class decisionTable:
    def __init__(self):
        self.movesTable = {}
        # each element: ("current board state - suggested move"[str] : good/bad value [int])
        self.epsilon = 1.0
        self.movesPath = []

    def add2path(self,board,move):
        item = str(board.board) + "-" + str(move)
        self.movesPath += [item]

    def updateMovesTable(self,winner):
        if winner[0].upper() == 'X':
            resultNum = 1
        elif winner[0].upper() == 'O':
            resultNum = -1
        else:
            resultNum = 0
        for x in self.movesPath:
            if x in self.movesTable:
                self.movesTable[x] += resultNum
            else:
                self.movesTable[x] = resultNum
        self.movesPath = []

In [ ]:
def makeMove(board,player):
    global myTable

    #print("epsilon = "+str(myTable.epsilon),end="")
    if random.random() < myTable.epsilon:
    #if False:
        #print("\tMaking random move...")
        makeRandomMove(board,player)
    else:
        #print("\tMaking calculated move...")
        makeCalculatedMove(board,player)
    myTable.epsilon = myTable.epsilon * 0.999999

def makeRandomMove(board,player):
    global myTable

    moves = board.availableSquares()
    item = random.randint(0,len(moves)-1)
    myTable.add2path(board,moves[item])
    board.makeMoveComputer(player,moves[item])

def makeCalculatedMove(board,player):
    global myTable

    moves = board.availableSquares()
    currentBoard = str(board.board)
    bestMoveVal = -99999
    bestMove = ""
    for x in moves:
        testMove = currentBoard+"-"+str(x)
        if testMove in myTable.movesTable.keys():
            testMoveVal = myTable.movesTable[testMove]
            if player == 'O':
                testMoveVal = testMoveVal * -1
        else:
            testMoveVal = 0
        if testMoveVal > bestMoveVal:
            bestMove = x
            bestMoveVal = testMoveVal
        elif testMoveVal == bestMoveVal:
            if random.random() > .5:
                bestMove = x
    myTable.add2path(board,bestMove)
    board.makeMoveComputer(player,bestMove)

In [ ]:
def playOneGame():
    global movesPath

    myBoard = tictactoeBoard()
    done = False
    player = 'X'
    if numIterations == 1:
        myBoard.printBoard()
    while not done:
        if (player == 'X' and playerXtype == "H") or (player == 'O' and playerOtype == "H"):
            myMove = input(player+"'s move: ")
            myBoard.makeMoveHuman(player,myMove)
        else:
            makeMove(myBoard,player)
        if numIterations == 1:
            myBoard.printBoard()
        if myBoard.isWin(player):
            if numIterations == 1:
                print(player+ " wins!")
            done = True
            result = player
        elif myBoard.isDraw():
            if numIterations == 1:
                print("It's a draw.")
            done = True
            result = 'D'
        else:
            if player == 'X':
                player = 'O'
            else:
                player = 'X'
    return result

In [ ]:
# load (or make new) table
if exists(tablePath):
    with open(tablePath, 'rb') as handle:
        myTable = pickle.load(handle)
else:
    myTable = decisionTable()

# play the game
numIterations = int(input("Number of iterations? "))
if numIterations == 1:
    playerXtype = input("Player X (h)uman or (c)omputer? ")[0].upper()
    playerOtype = input("Player O (h)uman or (c)omputer? ")[0].upper()
    #print(playerXtype)
else:
    playerXtype = "C"
    playerOtype = "C"
Xwins = 0
Owins = 0
draws = 0
start = time.time()
for x in range(0,numIterations):
    winner = playOneGame()
    if winner=='X':
        Xwins += 1
    elif winner=='O':
        Owins += 1
    else:
        draws += 1
    myTable.updateMovesTable(winner)
end = time.time()
print("\nEpsilon = "+str(myTable.epsilon)+"\n")
print("X wins: "+str(Xwins),end="")
print(" (%5.2f%%)" % (100.0*Xwins/numIterations))
print("O wins: "+str(Owins),end="")
print(" (%5.2f%%)" % (100.0*Owins/numIterations))
print("Draws: "+str(draws),end="")
print(" (%5.2f%%)" % (100.0*draws/numIterations))
print("\nTime to run: %.2f seconds" % (end-start))

# save updated move table w/ new info
with open(tablePath, 'wb') as handle:
    pickle.dump(myTable, handle, protocol=pickle.HIGHEST_PROTOCOL)